In [1]:
import pandas as pd
import numpy as np

In [5]:
poll_analysis_experience = '/Users/churnika/Desktop/Projects/ML-Crate/LinkedIn Poll Data Analysis/Dataset/LIQuizPollData%20-%20Sheet1%20(2).csv'
poll_data = pd.read_csv(poll_analysis_experience)
poll_data = poll_data.dropna()
poll_data.describe()

,Quiz_number,Total_Views,Total_Responses,Right_Answers,Total_Likes
count,100.000000,100.000000,100.000000,100.000000,100.000000
mean,50.500000,60214.630000,874.240000,509.360000,36.110000
std,29.011492,26489.886212,756.884111,509.713319,18.822346
min,1.000000,24307.000000,157.000000,37.000000,10.000000
25%,25.750000,44482.750000,448.250000,191.500000,24.000000
50%,50.500000,52106.500000,638.500000,389.500000,32.000000
75%,75.250000,72124.000000,1026.250000,661.000000,43.500000
max,100.000000,172686.000000,5669.000000,4093.000000,139.000000
